In [ ]:
import torch
import torch.nn as nn
import numpy as np


In [6]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [7]:
class GAN(nn.Module):
  def __init__(self, input_dim, hidden_dim, output_dim):
    super(GAN, self).__init__()
    
    # Generator
    self.generator = nn.Sequential(
      nn.Linear(input_dim, hidden_dim),
      nn.LeakyReLU(0.2),
      nn.BatchNorm1d(hidden_dim),
      nn.Linear(hidden_dim, hidden_dim * 2),
      nn.LeakyReLU(0.2),
      nn.BatchNorm1d(hidden_dim * 2),
      nn.Linear(hidden_dim * 2, output_dim),
      nn.Tanh()
    )
    
    # Discriminator
    self.discriminator = nn.Sequential(
      nn.Linear(output_dim, hidden_dim * 2),
      nn.LeakyReLU(0.2),
      nn.Dropout(0.3),
      nn.Linear(hidden_dim * 2, hidden_dim),
      nn.LeakyReLU(0.2),
      nn.Dropout(0.3),
      nn.Linear(hidden_dim, 1),
      nn.Sigmoid()
    )
  
  def g_forward(self, x):
    x = self.generator(x)
    x = x.view(x.size(0), -1)
    return x
  
  def d_forward(self, x):
    x = x.view(x.size(0), -1)
    x = self.discriminator(x)
    return x
  
  
  

In [10]:
# Initialize the GAN model
input_dim = 100  # Dimension of random noise input
hidden_dim = 128
output_dim = 784  # Example: for 28x28 images

model = GAN(input_dim, hidden_dim, output_dim)

# Loss function
criterion = nn.BCELoss().to(device)

# Optimizers - separate optimizers for generator and discriminator
lr = 0.0002
optimizer_G = torch.optim.Adam(model.generator.parameters(), lr=lr, betas=(0.5, 0.999))
optimizer_D = torch.optim.Adam(model.discriminator.parameters(), lr=lr, betas=(0.5, 0.999))

In [16]:
# Training loop
num_epochs = 100
batch_size = 64

model.to(device)

for epoch in range(num_epochs):
  # Generate fake data
  z = torch.randn(batch_size, input_dim).to(device)
  fake_data = model.g_forward(z)
  
  # Train Discriminator
  optimizer_D.zero_grad()
  
  # Real data (you need to replace this with actual data)
  real_data = torch.randn(batch_size, output_dim).to(device)
  real_labels = torch.ones(batch_size, 1).to(device)
  fake_labels = torch.zeros(batch_size, 1).to(device)
  
  # Discriminator loss on real data
  real_output = model.d_forward(real_data)
  d_loss_real = np.log(real_output).mean()
  
  # Discriminator loss on fake data
  fake_output = model.d_forward(fake_data.detach())
  d_loss_fake = np.log(1 - fake_output).mean()
  
  # Total discriminator loss
  d_loss = d_loss_real + d_loss_fake
  d_loss.backward()
  optimizer_D.step()
  
  # Train Generator
  optimizer_G.zero_grad()
  
  # Generator loss
  fake_output = model.d_forward(fake_data)
  g_loss = np.log(fake_output).mean()
  g_loss = np.log(1 - fake_output).mean()
  g_loss.backward()
  optimizer_G.step()
  
  if (epoch + 1) % 10 == 0:
    print(f'Epoch [{epoch+1}/{num_epochs}], d_loss: {d_loss.item():.4f}, g_loss: {g_loss.item():.4f}')

RuntimeError: Can't call numpy() on Tensor that requires grad. Use tensor.detach().numpy() instead.